In [1]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

In [ ]:
fake = Faker('pt_BR')  # Dados em português

# Conexão com o Postgres
conn = psycopg2.connect(
    dbname="dw_vendas_dbt",
    user="seu_usuario",
    password="sua_senha",
    host="localhost"
)
cursor = conn.cursor()



In [ ]:
# ---  Ingestão de LOJAS  ---
nomes_lojas = ["Super", "Mega", "Hiper", "Ultra", "Maxi"]
for i in range(1, 21):  # 20 lojas
    cursor.execute(
        """
        INSERT INTO lojas_bronze (loja_id, nome, cidade, tamanho)
        VALUES (%s, %s, %s, %s)
        """,
        (i,  # ID sequencial
         f"{random.choice(nomes_lojas)} {fake.word().capitalize()}",  # Nome composto
         fake.city(),
         random.choice(["Pequena", "Média", "Grande"]))
    )

In [ ]:
# --- Ingestão de CLIENTES ---
for i in range(1, 101):  # 100 clientes
    cursor.execute(
        """
        INSERT INTO clientes_bronze (cliente_id, nome, cidade, segmento)
        VALUES (%s, %s, %s, %s)
        """,
        (i,  # ID sequencial para evitar duplicatas
         fake.name(),
         fake.city(),
         random.choice(["Varejo", "Atacado", "Online"]))
    )

In [ ]:
# --- Ingestão de PRODUTOS ---
categorias = ["Eletrônicos", "Roupas", "Alimentos", "Livros", "Cosméticos"]
for i in range(1, 51):  # 50 produtos
    cursor.execute(
        """
        INSERT INTO produtos_bronze (produto_id, nome, categoria, preco)
        VALUES (%s, %s, %s, %s)
        """,
        (i,  # ID sequencial
         f"{fake.word().capitalize()} {fake.word()}",  # Nome composto
         random.choice(categorias),
         round(random.uniform(10, 500), 2))
    )

In [ ]:
# --- Ingestão de VENDAS ---
for i in range(1, 1001):  # 1000 vendas
    data_venda = fake.date_between(start_date='-1y', end_date='today')
    cursor.execute(
        """
        INSERT INTO vendas_bronze (venda_id, loja_id, cliente_id, produto_id, quantidade, data_venda, total)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (i,  # ID sequencial
         random.randint(1, 20),  # loja_id válido (1-20)
         random.randint(1, 100),  # cliente_id válido (1-100)
         random.randint(1, 50),  # produto_id válido (1-50)
         random.randint(1, 5),
         data_venda,
         round(random.uniform(50, 2000), 2))
    )

In [10]:
# Commit e fechamento
conn.commit()
cursor.close()
conn.close()
print("Dados ingeridos na Bronze com sucesso!")

Dados ingeridos na Bronze com sucesso!
